# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag, ne_chunk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.base import BaseEstimator, TransformerMixin


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)
X = df.message.values
y = df.iloc[:, 4:].values
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # punctuation removal
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # split into words
    tokens = word_tokenize(text)
    # stop words removal, normalize and remove space
    tokens = [w.lower().strip() for w in tokens if w not in stopwords.words("english")]
    # lemmatization
    tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
labels = df.iloc[:, 4:].columns.tolist()
for i in range(36):
    print(labels[i])
    print(classification_report(y_test[:, i], y_pred[:,i]))
    print('\n')

related
             precision    recall  f1-score   support

          0       0.65      0.49      0.56      1210
          1       0.85      0.92      0.88      3979
          2       0.31      0.33      0.32        40

avg / total       0.80      0.81      0.80      5229



request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      4339
          1       0.80      0.47      0.59       890

avg / total       0.88      0.89      0.88      5229



offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5200
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5229



aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3051
          1       0.75      0.60      0.67      2178

avg / total       0.75      0.75      0.75      5229



medical_help
             precision    recall

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Try using RidgeClassifier
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RidgeClassifier()))
])

In [16]:
pipeline3.fit(X_train, y_train)
y_pred_3 = pipeline3.predict(X_test)

In [17]:
for i in range(36):
    print(labels[i])
    print(classification_report(y_test[:, i], y_pred_3[:,i]))
    print('\n')

related
             precision    recall  f1-score   support

          0       0.68      0.47      0.56      1210
          1       0.85      0.93      0.89      3979
          2       0.83      0.12      0.22        40

avg / total       0.81      0.82      0.81      5229



request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      4339
          1       0.81      0.58      0.67       890

avg / total       0.90      0.90      0.90      5229



offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5200
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5229



aid_related
             precision    recall  f1-score   support

          0       0.78      0.83      0.80      3051
          1       0.74      0.66      0.70      2178

avg / total       0.76      0.76      0.76      5229



medical_help
             precision    recall

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# create entity-recognizer

class NamedEntityExtractor(BaseEstimator, TransformerMixin):

    def named_entity(self, text):
        tree = ne_chunk(pos_tag(tokenize(text)))
        if 'NNP' in str(tree):
            return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_NNP = pd.Series(X).apply(self.named_entity)
        return pd.DataFrame(X_NNP)

In [23]:
pipeline4 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('named_entity', NamedEntityExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [24]:
pipeline4.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_...', smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('named_entity', NamedEntityExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [33]:
parameters = {
    #'features__text_pipeline__vect__ngram_range':[(1, 1), (1, 2)],
    #'tfidf__smooth_idf': [True, False],
    # 'clf__estimator__n_estimators': [10, 30]
    #'clf__estimator__max_features': [2, 3, 4]
    'features__text_pipeline__tfidf__norm': ['l1', 'l2'],
    #'features__text_pipeline__tfidf__smooth_idf': [True, False],
    'clf__estimator__bootstrap': [True, False]
}

cv = GridSearchCV(pipeline4, param_grid=parameters, verbose=3, cv=2, n_jobs=-1)

In [34]:
cv.fit(X_train, y_train)
y_pred_4 = cv.predict(X_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l1 
[CV]  clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l1, score=0.23379231210556511, total= 6.0min
[CV] clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l1 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.8min remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l1, score=0.22769436740939084, total= 5.9min
[CV] clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l2 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 17.5min remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l2, score=0.2381908586727864, total= 6.0min
[CV] clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l2 
[CV]  clf__estimator__bootstrap=True, features__text_pipeline__tfidf__norm=l2, score=0.22951133212202351, total= 5.9min
[CV] clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l1 
[CV]  clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l1, score=0.23637406769936892, total= 6.2min
[CV] clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l1 
[CV]  clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l1, score=0.224538586592713, total= 6.1min
[CV] clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l2 
[CV]  clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l2, score=0.23493975903614459, total= 6.2min
[CV] clf__estimator__bootstrap=False, features__text_pipeline__tfidf__norm=l2 
[CV]  

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 70.7min finished


In [35]:
for i in range(36):
    print(labels[i])
    print(classification_report(y_test[:, i], y_pred_4[:,i]))
    print('\n')

related
             precision    recall  f1-score   support

          0       0.62      0.47      0.53      1210
          1       0.85      0.91      0.88      3979
          2       0.23      0.23      0.23        40

avg / total       0.79      0.80      0.79      5229



request
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      4339
          1       0.82      0.44      0.57       890

avg / total       0.88      0.89      0.87      5229



offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5200
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5229



aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3051
          1       0.75      0.60      0.67      2178

avg / total       0.75      0.75      0.74      5229



medical_help
             precision    recall

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.